http://www.cs.cmu.edu/~xuerui/papers/ctr.pdf

In [14]:
from scipy.special import psi
import numpy as np

In [6]:
x = [(100, 70)]

In [8]:
for i, c in  x:
    print(i, c)

100 70


In [10]:
def hyperprior(x, a, b, loop=1000):
    for l in range(loop):
        a_t = 0
        a_b = 0
        b_t = 0
        for (i, c) in x:
            a_t = a_t + psi(c+a) - psi(a)
            a_b = a_b + psi(i+a+b) - psi(a+b)
            b_t = b_t + psi(i-c+b) - psi(b)
        a = a_t / a_b * a
        b = b_t / a_b * b
        print((l, a,b, a/(a+b)))

In [13]:
hyperprior(x, 1, 2)

(0, 1.3036766378408218, 1.6332224886632591, 0.4438956129190058)
(1, 1.5458644923199623, 1.4373399964793945, 0.5181892485493419)
(2, 1.7420319425781308, 1.3375606424955, 0.5656696119550113)
(3, 1.9068219171252463, 1.2936482159685496, 0.5957943170311608)
(4, 2.051022172196233, 1.2833079338514413, 0.6151227103987608)
(5, 2.1819944760611345, 1.2937111360945615, 0.6277846053560968)
(6, 2.304628749267076, 1.3172153389032164, 0.6363136273023127)
(7, 2.4221398966978436, 1.3491412205592934, 0.6422591743729442)
(8, 2.536632166235798, 1.3865664932169102, 0.6465724492752711)
(9, 2.6494791074442454, 1.4276414386838265, 0.6498407583166463)
(10, 2.7615749674815473, 1.4711850108352893, 0.6524289072917598)
(11, 2.873500031994767, 1.5164390768169564, 0.6545648950407813)
(12, 2.985629151850003, 1.5629154932340883, 0.6563921836134481)
(13, 3.098202924726719, 1.6102994645613542, 0.6580017739345712)
(14, 3.211374356969954, 1.658387844138014, 0.6594519864315556)
(15, 3.32523942981517, 1.707049492455962, 0.66

この方曰く、論文の収束条件(1000回ループ)では全く収束できなさそうとのこと

加えて、$\alpha' = \alpha \frac{X}{Y}$のXとYが非常に近い時、X/Yが1に丸められる丸め誤差があると指摘

以下に式変換して、XとYの差を計算することで丸められずに更新を進めることができた

$\alpha' = \alpha \frac{X}{Y} = \alpha + \alpha\frac{X-Y}{Y} = \alpha + \alpha \frac{\sum(x - y)}{\sum y}$

In [32]:
def new_hyperprior(x, a, b, loop=1000):
    for l in range(loop):
        a_t__a_b = 0
        a_b = 0
        b_t__a_b = 0
        for (i, c) in x:
            if c != 0:
                a_t__a_b += psi(c+a) - psi(a)
            a_b_ = psi(i+a+b) - psi(a+b)
            a_b += a_b_
            a_t__a_b -= a_b_
            b_t__a_b += psi(i-c+b) - psi(b)
            b_t__a_b -= a_b_
        a_r = a_t__a_b / a_b
        a = a_r * a + a
        b_r = b_t__a_b / a_b
        b = b_r * b + b
        if (l % 1000 == 0):
            print(f'l:{l}, a:{a: .15f}, b:{b: .15f}')
        # if (np.abs(a_r) < EPSILON) & (np.abs(b_r) < EPSILON): EPSILON変数がどこで作られてるのか不明なためスルー 両方誤差より小さければ実行を打ち止めするものっぽい
        #     break
    print(f'l:{l} a:{a: .15f}, b:{b: .15f}')

結果ほぼ変わらず？

In [33]:
new_hyperprior(x, 1, 2)

l:0, a: 1.303676637840822, b: 1.633222488663259
l:999 a: 258.706564708854842, b: 111.159791543251600


## その他推定手法
- ベータ分布の平均と分散に合わせる

In [34]:
def hyperprior_est_beta(a, b):
    mu = a / (a + b)
    sigma2 = (a*b) / (((a*b)**2) * (a + b + 1))
    alpha = - ((mu*sigma2) + mu**3 - mu**2) / sigma2
    beta = (((mu - 1)* sigma2) + mu**3 - (2 * mu**2) + mu) / sigma2
    return alpha, beta

In [36]:
hyperprior_est_beta(70, 30)

(31178.000000000007, 13361.999999999998)